### Daniel Behar Aldana, 20200402
### LAB 3

In [24]:
import pandas as pd

In [25]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [26]:
host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


### Parte 1: Preguntas generales de SQL:

In [27]:
# 1.Obtener el nombre, apellido, salario, título y vigencia del mismo para el empleado 12557. 
# Su resultado se debe de ver de la siguiente manera:

In [28]:
query1 = """
SELECT e.first_name, e.last_name, e.emp_no, t.title, s.from_date, s.to_date, s.salary
FROM employee.employees e
JOIN employee.titles t ON e.emp_no = t.emp_no
JOIN employee.salaries s ON e.emp_no = s.emp_no
WHERE e.emp_no = 12557
"""
tables = pd.read_sql(query1,connection)
tables

,first_name,last_name,emp_no,title,from_date,to_date,salary
0,Stafford,Kopetz,12557,Senior Engineer,1988-03-20,1989-03-20,41840
1,Stafford,Kopetz,12557,Senior Engineer,1989-03-20,1990-03-20,46021
2,Stafford,Kopetz,12557,Senior Engineer,1990-03-20,1991-03-20,48699
3,Stafford,Kopetz,12557,Senior Engineer,1991-03-20,1992-03-19,51610
4,Stafford,Kopetz,12557,Senior Engineer,1992-03-19,1993-03-19,51351
5,Stafford,Kopetz,12557,Senior Engineer,1993-03-19,1994-03-19,51660
6,Stafford,Kopetz,12557,Senior Engineer,1994-03-19,1995-03-19,52878
7,Stafford,Kopetz,12557,Senior Engineer,1995-03-19,1996-03-18,54053
8,Stafford,Kopetz,12557,Senior Engineer,1996-03-18,1997-03-18,57950
9,Stafford,Kopetz,12557,Senior Engineer,1997-03-18,1998-03-18,58055


In [29]:
# 2.Encontrar el último departamento en el que trabajó el empleado con la siguiente información del mismo: 
# nombre, apellido, número de empleado, fecha de nacimiento, código de departamento, nombre del departamento, 
# fecha cuando inició su trabajo en ese departamento y la fecha cuando concluyó su trabajo

In [30]:
query2 = """
SELECT e.first_name, e.last_name, birth_date, e.emp_no, t.title, s.from_date, s.to_date
FROM employee.employees e
LEFT JOIN employee.titles t ON e.emp_no = t.emp_no
LEFT JOIN employee.salaries s ON e.emp_no = s.emp_no
WHERE e.emp_no = 12557
ORDER BY s.from_date DESC limit 1;
"""
tables = pd.read_sql(query2,connection)
tables

,first_name,last_name,birth_date,emp_no,title,from_date,to_date
0,Stafford,Kopetz,1954-01-01,12557,Senior Engineer,2002-03-17,9999-01-01


In [31]:
# 3.Encontrar el salario máximo de cada empleado utilizando la función Partition By con toda la información del empleado
# y de su salario

In [32]:
query3 = """
SELECT *
FROM (
    SELECT e.emp_no, e.birth_date, e.first_name, e.last_name, e.gender, s.salary
    , ROW_NUMBER() OVER (PARTITION BY e.emp_no ORDER BY s.salary DESC) AS seq
    FROM employee.employees e
    LEFT JOIN employee.salaries s ON s.emp_no = e.emp_no
    GROUP BY 1,2,3,4,5,6
    ) A
WHERE seq = 1
LIMIT 100
"""
tables = pd.read_sql(query3,connection)
tables

,emp_no,birth_date,first_name,last_name,gender,salary,seq
0,10001,1953-09-02,Georgi,Facello,M,88958,1
1,10002,1964-06-02,Bezalel,Simmel,F,72527,1
2,10003,1959-12-03,Parto,Bamford,M,43699,1
3,10004,1954-05-01,Chirstian,Koblick,M,74057,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,94692,1
...,...,...,...,...,...,...,...
95,10096,1954-09-16,Jayson,Mandell,M,68612,1
96,10097,1952-02-27,Remzi,Waschkowski,M,70161,1
97,10098,1961-09-23,Sreekrishna,Servieres,F,56202,1
98,10099,1956-05-25,Valter,Sullins,F,98538,1


### Parte 2: Asesinato:

In [37]:
import sqlite3
con = sqlite3.connect('sql-murder-mystery.db')
cursor = con.cursor()

In [46]:
#Nombres de las columnas e info de nombre 1
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('crime_scene_report',), ('drivers_license',), ('person',), ('facebook_event_checkin',), ('interview',), ('get_fit_now_member',), ('get_fit_now_check_in',), ('income',), ('solution',)]


In [48]:
#Testimonial
queryInfo = """
SELECT description
FROM crime_scene_report
WHERE lower(city) = 'sql city'
AND date = 20180115
AND lower(type) = 'murder';
"""
table = pd.read_sql(queryInfo, con)
table["description"][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [51]:
#Testigo 1
queryW1 = """
SELECT *
FROM person
WHERE lower(address_street_name) = 'northwestern dr'
AND address_number = (SELECT MAX(address_number)
    FROM person 
    WHERE lower(address_street_name) = 'northwestern dr');
"""
pd.read_sql(queryW1 , con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [52]:
#Testigo 2
queryW2 = """
SELECT *
FROM person
WHERE lower(name) like 'annabel%'
AND lower(address_street_name) = 'franklin ave';
"""
pd.read_sql(queryW2, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [53]:
#Testimonial testigo 2
queryT1 = """
SELECT *
FROM interview
WHERE person_id = (SELECT id
    FROM person
    WHERE lower(name) = 'annabel miller');
"""
pd.read_sql(queryT1, con)
table = pd.read_sql(queryT1, con)
table["transcript"][0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [55]:
#Testimonial testigo 2
queryT2 = """
SELECT *
FROM interview
WHERE person_id = (SELECT id
    FROM person
    WHERE lower(name) = 'morty schapiro');
"""
table = pd.read_sql(queryT2, con)
table["transcript"][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [58]:
#Info del Gym PT1
queryG1 = """
SELECT *
FROM get_fit_now_member
WHERE lower(membership_status) = 'gold'
AND upper(id) like '48Z%';
"""
pd.read_sql(queryG1, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [59]:
#Info del Gym PT2
queryG2 = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109
AND upper(membership_id) like '48Z%';
"""
pd.read_sql(queryG2, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [60]:
#Info del carro
queryCar = """
SELECT *
FROM drivers_license
WHERE upper(plate_number) like '%H42W%' ;
"""
pd.read_sql(queryCar, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [61]:
#Teorias
queryPos = """
SELECT *
FROM person
WHERE id in (SELECT person_id
    FROM get_fit_now_member
    WHERE lower(membership_status) = 'gold'
    AND lower(id) like '48z%');
"""
pd.read_sql(queryPos, con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [69]:
#Probable asesino
queryA = """
SELECT d.*, p.id, p.name
FROM drivers_license d
JOIN person p on p.license_id = d.id
WHERE d.id in (SELECT license_id
    FROM person
    WHERE id in (SELECT person_id
        FROM get_fit_now_member
        WHERE lower(membership_status) = 'gold'
        AND upper(id) like '48Z%'));
"""
pd.read_sql(queryA, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers


In [68]:
#El asesino es Jeremy Bowers

In [63]:
#Verificacion acertada
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [70]:
#Transcripcion asesino
queryTa = """
SELECT *
FROM interview
WHERE person_id = (SELECT id
    FROM person
    WHERE lower(name) = 'jeremy bowers');
"""
table = pd.read_sql(queryTa, con)
table["transcript"][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [71]:
#Buscando autora intelectual
queryAut = """
SELECT * 
FROM person
WHERE id = (SELECT distinct(person_id) 
    FROM facebook_event_checkin
    WHERE date like '201712%'
    AND lower(event_name) = 'sql symphony concert'
    AND person_id in (SELECT id
        FROM person
        WHERE license_id in (SELECT id
            FROM drivers_license
            WHERE lower(car_model) = 'model s'
            AND height between 65 and 67
            AND lower(gender) = 'female')));
"""
pd.read_sql(queryAut, con)

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [ ]:
#La autora intelectual es Miranda Priestly

In [72]:
#Verificacion 2 acertada
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'